#### Notes
- process of elimination is used to wittle out parcels
- owned unit groupings (OUG) relies parcel common areas
    - if those aren't are accurately attributed, parcels within won't be correctly attributed
    
#### uncategorized tpyes
- 106
- 510 Comm Imps in Res Zone
- 911 vacant lots
- 957 Related Parcel
- 997 Residential - Multi

In [1]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

In [2]:
# tag_expression and code block for tagging parcels
tag_expression = "createTag(!TAG!, '{}')"
tag_codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

In [3]:
# input taz shapefile and parcels
taz_shp = '.\\Inputs\\TAZ.shp'
parcels = '.\\Inputs\\Utah_Salt_Lake_County_Parcels_LIR.gdb\\Parcels_SaltLake_LIR_UTM12'

In [4]:
# create output gdb
outputs = '.\\Outputs'
gdb = os.path.join(outputs, "saltlake4.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "saltlake4.gdb")

In [5]:
# select parcels within modeling area
parcels_layer = arcpy.MakeFeatureLayer_management(parcels, 'parcels') 
arcpy.SelectLayerByLocation_management(parcels_layer, 'HAVE_THEIR_CENTER_IN', taz_shp)
parcels_for_modeling = arcpy.FeatureClassToFeatureClass_conversion(parcels_layer, gdb, '_01_parcels_for_modeling')


# add a tag field for parcel type
arcpy.AddField_management(parcels_for_modeling, 'TAG', 'TEXT')
arcpy.AddField_management(parcels_for_modeling, 'TYPE_WFRC', 'TEXT')
arcpy.AddField_management(parcels_for_modeling, 'SUBTYPE_WFRC', 'TEXT')

# count parts and rings
arcpy.AddField_management(parcels_for_modeling, 'PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling, 'RINGS', 'LONG')

fields = ["shape@", 'PARTS', 'RINGS']
with arcpy.da.UpdateCursor(parcels_for_modeling, fields) as cursor:
    for row in cursor:
        shape = row[0]
        parts = shape.partCount
        rings = shape.boundary().partCount   
        row[1] = parts
        row[2] = rings
        cursor.updateRow(row)


# create the master layer for selection
parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(parcels_for_modeling, 'parcels_for_modeling_layer')

# get a count of all parcels
count_all = arcpy.GetCount_management(parcels_for_modeling_layer)
print("# intial parcels in modeling area:\n {}".format(count_all))

# # create centroids for all parcels
# parcels_for_modeling_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
#                                                                  os.path.join(gdb, '_01b_parcels_for_modeling_centroids'), 
#                                                                  "INSIDE")     

# intial parcels in modeling area:
 384322


In [6]:
###########
# churches
###########

query = (""" PROP_CLASS = 'ER' OR PROP_TYPE =  '956' """)
tag="churches"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

928 "churches" parcels were selected.
383394 parcels remain...


In [7]:
###############
# agriculture
###############

query = """ PROP_TYPE IN ('811','812','816','817', '818','830','850') """
tag="agriculture"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1299 "agriculture" parcels were selected.
382095 parcels remain...


In [8]:
###############
# open space
###############

# get parcels on parks
parks = ".\\Inputs\\ParksLocal.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=parks,
                                       selection_type='NEW_SELECTION')

# get parcels on public lands that are not military
land_ownership_lyr = arcpy.MakeFeatureLayer_management('./Inputs/UT_SITLA_Ownership_LandOwnership_WM.shp','land_ownership_lyr') 

query= (""" OWNER <> 'Private' And DESIG <> 'Military' """)
arcpy.SelectLayerByAttribute_management(land_ownership_lyr, 'NEW_SELECTION', query)

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=land_ownership_lyr,
                                       selection_type='ADD_TO_SELECTION')

# get cemetery parcels using overlay
cemeteries_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\Cemeteries.shp','cemeteries_lyr') 
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=cemeteries_lyr,
                                       selection_type='ADD_TO_SELECTION')

# get parcels that are golf courses using overlay
golf_courses = ".\\Inputs\\GolfCourses.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=golf_courses,
                                       selection_type='ADD_TO_SELECTION')

# query parcel attributes
query= (""" PROP_TYPE IN ('951','960','961') OR TAXEXEMPT_TYPE = 'CM' """)
tag = "open_space"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

7085 "open_space" parcels were selected.
375010 parcels remain...


In [9]:
###############
# health care
###############

query = (""" PROP_TYPE in ('524','547','560','561') """)
tag = 'healthcare'

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

415 "healthcare" parcels were selected.
374595 parcels remain...


In [10]:
###############
# education
###############

query = (""" PROP_TYPE =  '954' OR PROP_TYPE =  '577' OR TAXEXEMPT_TYPE = 'EE' """)
tag="education"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

684 "education" parcels were selected.
373911 parcels remain...


In [11]:
###############
# government
###############

query = (""" PROP_TYPE IN ('570','953','955') """)
tag="government"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

5297 "government" parcels were selected.
368614 parcels remain...


In [12]:
###############
# retail
###############
query=(""" PROP_TYPE IN ('513','514','515','516','528','529','530','536','548','549','551','553','562','564','573','574',
            '575','582','583','584','585','591','675','749','775','914') """)
tag="retail"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

5475 "retail" parcels were selected.
363139 parcels remain...


In [13]:
###############
# industrial
###############

query = (""" PROP_TYPE IN ('200','202','203', '556','695','795','915') OR PROP_CLASS = 'I' """)
tag="industrial"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

6793 "industrial" parcels were selected.
356346 parcels remain...


In [14]:
###############
# office
###############

query= ("""PROP_TYPE IN ('566','590','760','916') """)
tag="office"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

2375 "office" parcels were selected.
353971 parcels remain...


In [15]:
################
# single_family
################

query = (''' PROP_TYPE IN ('103','104','111', '118', '888','994','998') ''')
tag="single_family"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))


#should count address points in resulting features and look for duplexes/mh

231057 "single_family" parcels were selected.
122914 parcels remain...


In [17]:
########################
# single_family w/ ADU
########################

query = (''' PROP_TYPE IN ('106') ''')
tag="single_family"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'single_family_adu'",
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))


#should count address points in resulting features and look for duplexes/mh

1446 "single_family" parcels were selected.
121468 parcels remain...


In [18]:
################
# duplexes
################

query = (''' PROP_TYPE IN ('112') ''')
tag="duplex"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

5396 "duplex" parcels were selected.
116072 parcels remain...


In [19]:
###############
# utilities
###############

# select parcels using query
query = (''' PROP_TYPE IN ('950') ''')
tag="utilities"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

10 "utilities" parcels were selected.
116062 parcels remain...


In [20]:
#################
# group quarters
#################

query= (""" PROP_TYPE IN ('105','540','576') """)
tag = "group_quarters"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

143 "group_quarters" parcels were selected.
115919 parcels remain...


In [21]:
#################
# Apartments
#################

# reorder two processes

query= (""" PROP_TYPE IN ('110','113','114', '115', '120', '150', '199') """)
tag = "apartments"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = int(arcpy.GetCount_management(parcels_for_modeling_layer)[0])

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'PROP_TYPE'",
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)



##############

query = """ PROP_TYPE IN ('713') """

# filter for common areas
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = count_type + int(arcpy.GetCount_management(parcels_for_modeling_layer)[0])

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

common_areas =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_05a_apt_common_areas')

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# fill in holes via eliminate polygon part
common_areas_filled = arcpy.EliminatePolygonPart_management(common_areas, os.path.join(gdb, '_05b_apt_common_areas_filled'),
                                                      condition='PERCENT', part_area_percent=60, part_option='CONTAINED_ONLY')
# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=common_areas_filled, selection_type='NEW_SELECTION')

# count the selected features
count_type = count_type + int(arcpy.GetCount_management(parcels_for_modeling_layer)[0])

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# convert condo parcels that are contained by common areas into centroids
condo_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(gdb, '_05c_apt_centroids'), "INSIDE")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# use spatial join to summarize market value & acreage
target_features = common_areas_filled
join_features = condo_centroids
output_features = os.path.join(gdb, "_05d_apt_oug_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# parcel acres
fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "CONTAINS")



##############

# get count from address points

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

5294 "apartments" parcels were selected.
110625 parcels remain...


In [22]:
#################
# OUG - Condos
#################

tag = "condo"
query = """ PROP_TYPE IN ('711')) """

# filter for common areas, some small office parks and a few other strange cases may be present
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

common_areas =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_04a_condo_common_areas')

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# fill in holes via eliminate polygon part
common_areas_filled = arcpy.EliminatePolygonPart_management(common_areas, os.path.join(gdb, '_04b_condo_common_areas_filled'),
                                                      condition='PERCENT', part_area_percent=60, part_option='CONTAINED_ONLY')
# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=common_areas_filled, selection_type='NEW_SELECTION')

# count the selected features
count_type = int(count_type[0]) + int(arcpy.GetCount_management(parcels_for_modeling_layer)[0])

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# convert condo parcels that are contained by common areas into centroids
condo_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(gdb, '_04c_condos_centroids'), "INSIDE")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# use spatial join to summarize market value & acreage
target_features = common_areas_filled
join_features = condo_centroids
output_features = os.path.join(gdb, "_04d_condo_oug_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# parcel acres
fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "CONTAINS")


# use reverse grouping on remaining condo units

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

27602 "condo" parcels were selected.
83023 parcels remain...


In [23]:
#####################
# low income housing
#####################

query= (""" PROP_TYPE IN ('142') """)
tag = "low_income_housing"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
features = arcpy.EliminatePolygonPart_management(parcels_for_modeling_layer, os.path.join( gdb, '_02_{}'.format(tag)),
                                                      condition='PERCENT', part_area_percent=60, part_option='CONTAINED_ONLY')


# roundabout way to eliminate individual units inside low income common areas
layer = arcpy.MakeFeatureLayer_management(features, 'layer') 
query= (""" TAXEXEMPT_TYPE = 'MC' """)
arcpy.SelectLayerByAttribute_management(layer, 'NEW_SELECTION', query)
common_areas = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}_commons'.format(tag))
arcpy.SelectLayerByLocation_management(in_layer=layer,overlap_type="COMPLETELY_WITHIN",
                                       select_features=common_areas,
                                       selection_type='NEW_SELECTION')
layer = arcpy.DeleteFeatures_management(layer)

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# count address points in filled version

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

379 "low_income_housing" parcels were selected.
82644 parcels remain...


In [25]:
#################
# OUG - mobile home parks
#################

tag="mobile_home"

# use overlay to select mobile home parks parcels
mobile_home_parks = ".\\Inputs\\Mobile_Home_Parks.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=mobile_home_parks,
                                       selection_type='NEW_SELECTION')

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# count address points

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))


71 "mobile_home" parcels were selected.
82573 parcels remain...


In [26]:
#################
# commercial
#################

query=""" PROP_TYPE IN ('500','592', '593', '594', '595', '596') or PROP_CLASS = 'C' """
tag="commercial"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

8626 "commercial" parcels were selected.
73947 parcels remain...


In [27]:
#################
# roads
#################

query=""" BLDG_SQFT = 0 And FLOORS_CNT = 0 And TOTAL_MKT_VALUE = 0 And PARCEL_ADD IS NULL """
tag="road"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# tag the features
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=tag_expression.format(tag),
                                expression_type="PYTHON3",code_block=tag_codeblock)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

16302 "road" parcels were selected.
57645 parcels remain...


In [28]:
#################
# Vacant land
#################

query= (""" PROP_TYPE IN ('901','902','903', '905', '906', '908', '911', '917') """)
tag = "vacant"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

5970 "vacant" parcels were selected.
51675 parcels remain...


In [24]:
#################
# OUG - PUDs
#################

In [29]:
# export remaining parcels
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_09_unclassed_parcels')

<Result '.\\Outputs\\saltlake4.gdb\\_09_unclassed_parcels'>